In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("D:/data/Aaron/ProstateMRL/Code/Extraction/PatientKey.csv")

df = df[df["Treatment"] == "20fractions"]
list = df["PatID"].unique().astype(str)
array = np.asarray(list)
print(array)

FileNotFoundError: [Errno 2] No such file or directory: 'D:/data/Aaron/ProstateMRL/Code/Extraction/PatientKey.csv'

In [2]:
import pandas as pd
import numpy as np
import datetime

import sys
import os

current = os.path.dirname(os.path.realpath("Test.ipynb"))
parent = os.path.dirname(current)
sys.path.append(parent + "\\Functions\\")
import UsefulFunctions as UF
import ImageFunctions as IF

     PatID    Treatment      FileDir  Scan      Date
0      213  20fractions  20fractions   MR4  20110330
1      213  20fractions  20fractions  MR17  20110403
2      213  20fractions  20fractions  MR37  20110413
3      213  20fractions  20fractions  MR52  20110420
4      213  20fractions  20fractions  MR66  20110424
..     ...          ...          ...   ...       ...
192    829         SABR     SABR_new  MR15  20120219
193    955         SABR     SABR_new   MR1  20120408
194    955         SABR     SABR_new   MR6  20120411
195    955         SABR     SABR_new  MR16  20120415
196    955         SABR     SABR_new  MR22  20120418

[197 rows x 5 columns]
20fractions
213
   PatID    Treatment      FileDir  Scan      Date  Days
0    213  20fractions  20fractions   MR4  20110330     0
1    213  20fractions  20fractions  MR17  20110403     4
2    213  20fractions  20fractions  MR37  20110413    14
3    213  20fractions  20fractions  MR52  20110420    21
4    213  20fractions  20fractions  MR6

c:\Users\b01297ar\Documents\ProstateMRL-local\py-envs\PR-env\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\Users\b01297ar\Documents\ProstateMRL-local\py-envs\PR-env\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\Users\b01297ar\Documents\ProstateMRL-local\py-envs\PR-env\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro


752
     PatID Treatment FileDir  Scan      Date  Days
115    752      SABR    SABR   MR5  20120118     0
116    752      SABR    SABR   MR9  20120120     2
117    752      SABR    SABR  MR12  20120122     4
118    752      SABR    SABR  MR14  20120129    11
826
     PatID Treatment FileDir  Scan      Date  Days
119    826      SABR    SABR   MR5  20120201     0
120    826      SABR    SABR   MR8  20120203     2
121    826      SABR    SABR  MR11  20120205     4
122    826      SABR    SABR  MR14  20120208     7
123    826      SABR    SABR  MR17  20120210     9
1088
     PatID Treatment FileDir  Scan      Date  Days
124   1088      SABR    SABR  MR10  20120530     0
125   1088      SABR    SABR  MR13  20120601     2
126   1088      SABR    SABR  MR19  20120603     4
127   1088      SABR    SABR  MR25  20120606     7
1089
     PatID Treatment FileDir  Scan      Date  Days
128   1089      SABR    SABR   MR6  20120530     0
129   1089      SABR    SABR  MR12  20120601     2
130   1089  

c:\Users\b01297ar\Documents\ProstateMRL-local\py-envs\PR-env\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\Users\b01297ar\Documents\ProstateMRL-local\py-envs\PR-env\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\Users\b01297ar\Documents\ProstateMRL-local\py-envs\PR-env\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro